In [106]:
# from woocommerce import API
import pandas as pd
import numpy as np
import os

In [107]:
# from util.gsheet import update_sheet_by_id

sheet_id = "1ZE7BeE26iM43x_ympaIgOdl6lXWrwXg-PZZAKuIBEIw"
sheet_name = "OurStoreProducts"

# update_sheet_by_id(df, sheet_id, sheet_name)

In [108]:
# from util.gsheet import add_dropdown

# add_dropdown("OurStoreProducts", "list/delist", "List")

In [109]:
# import sys
# import os

# # In Jupyter, __file__ is not defined. Use the current working directory instead.
# sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [110]:
from util.gsheet import get_sheet_data

data = get_sheet_data(sheet_id, df=True, sheet_name=sheet_name)

In [111]:
data2 = get_sheet_data(sheet_id, df=True, sheet_name="SSI")


In [112]:
sdata = data2.copy()
sdata = sdata

In [113]:
sdata["SKU"]

0      1108232
1      1121607
2      1121608
3      1121599
4      1121604
        ...   
994           
995           
996           
997           
998           
Name: SKU, Length: 999, dtype: object

In [114]:
print(data.columns)
print(sdata.columns)
print(data['list/delist'].unique())

Index(['0', 'name', 'slug', 'permalink', 'date_created', 'date_created_gmt',
       'date_modified', 'date_modified_gmt', 'type', 'status', 'featured',
       'catalog_visibility', 'description', 'short_description', 'sku',
       'price', 'regular_price', 'sale_price', 'date_on_sale_from',
       'date_on_sale_from_gmt', 'date_on_sale_to', 'date_on_sale_to_gmt',
       'on_sale', 'purchasable', 'total_sales', 'virtual', 'downloadable',
       'download_limit', 'download_expiry', 'external_url', 'button_text',
       'tax_status', 'tax_class', 'manage_stock', 'stock_quantity',
       'backorders', 'backorders_allowed', 'backordered', 'low_stock_amount',
       'sold_individually', 'weight', 'shipping_required', 'shipping_taxable',
       'shipping_class', 'shipping_class_id', 'reviews_allowed',
       'average_rating', 'rating_count', 'parent_id', 'purchase_note',
       'menu_order', 'price_html', 'stock_status', 'has_options',
       'post_password', 'list/delist'],
      dtype='obje

In [115]:
# print(data[data['list/delist'] == 'List'].head())

In [116]:
# print(data[data['list/delist'] == 'Delist'])

In [118]:
def normalize_name(name):
    return str(name).strip().lower()


data['norm_name'] = data['name'].apply(normalize_name)
sdata['norm_name'] = sdata['Name'].apply(normalize_name)

# Merge on normalized name
comparison = pd.merge(
    sdata, data,
    on='norm_name',
    how='left',
    suffixes=('_ssi', '_store')
)


In [119]:
# Find matches and non-matches
matched = comparison[~comparison['sku'].isna()]
unmatched = comparison[comparison['sku'].isna()]

print("Matched products:", len(matched))
print("Unmatched SSI products:", len(unmatched))


Matched products: 3
Unmatched SSI products: 996


In [120]:
matched

,SKU,Name,Description,Price,Stock,Link,category,StoreStatus,norm_name,0,...,average_rating,rating_count,parent_id,purchase_note,menu_order,price_html,stock_status,has_options,post_password,list/delist
54,9004879,Mechanix Azimuth Tactical Combat Glove Black M...,Low-profile closure provies a secure fit to th...,$89.95,17,https://ssisports.net/Products/341112,Apparel,-,mechanix azimuth tactical combat glove black m...,19139,...,0.00,0,0,,0,"<del aria-hidden=""true""><span class=""woocommer...",instock,FALSE,,List
55,9004874,Mechanix Azimuth Tactical Combat Glove Black L...,Low-profile closure provies a secure fit to th...,$89.95,8,https://ssisports.net/Products/341107,Apparel,-,mechanix azimuth tactical combat glove black l...,19138,...,0.00,0,0,,0,"<span class=""woocommerce-Price-amount amount"">...",instock,FALSE,,List
56,4011677,Seirus HWS Workman Xtreme Mens Glove Gauntlet ...,"Puncture, cut, heat and tear resistant\nFive t...",$74.99,7,https://ssisports.net/Products/314419,Apparel,-,seirus hws workman xtreme mens glove gauntlet ...,23226,...,0.00,0,0,,0,"<span class=""woocommerce-Price-amount amount"">...",instock,FALSE,,List


In [ ]:
from config import SheetColumns
from util.sheet import get_attribute


def update_sheet_cell(spreadsheet, sheet_name, sku_list, stock_list):
    worksheet = spreadsheet.worksheet(sheet_name)
    
    # Get header row to find column index of 'SKU' and 'Stock'
    header = worksheet.row_values(1)
    
    # Get column names from config enum
    sku_col_name = get_attribute(sheet_name, SheetColumns.SKU)
    stock_col_name = get_attribute(sheet_name, SheetColumns.STOCK)
    
    try:
        sku_col_index = header.index(sku_col_name) + 1     # 1-indexed
        stock_col_index = header.index(stock_col_name) + 1 # 1-indexed
    except ValueError as e:
        print(f"Column not found: {e}")
        return

    for i, sku in enumerate(sku_list):
        try:
            # Find the SKU cell in the SKU column only
            cell = worksheet.find(str(sku), in_column=sku_col_index)
            
            # Get the current value of the cell
            current_value = worksheet.cell(cell.row, stock_col_index).value

            # Append the new value to the current value
            new_value = int(current_value)  + int(stock_list[i])
            # Update the cell with the new value
            worksheet.update_cell(cell.row, stock_col_index, new_value)
            print(f"Updated stock for SKU {sku} at row {cell.row}")
            
            
            # Update corresponding row in Stock column
            # worksheet.update_cell(cell.row, stock_col_index, stock_list[i])
            # print(f"Updated stock for SKU {sku} at row {cell.row}")
        
        except Exception as e:
            print(f"SKU {sku} not found. Error: {e}")


In [131]:
matched.columns

Index(['SKU', 'Name', 'Description', 'Price', 'Stock', 'Link', 'category',
       'StoreStatus', 'norm_name', '0', 'name', 'slug', 'permalink',
       'date_created', 'date_created_gmt', 'date_modified',
       'date_modified_gmt', 'type', 'status', 'featured', 'catalog_visibility',
       'description', 'short_description', 'sku', 'price', 'regular_price',
       'sale_price', 'date_on_sale_from', 'date_on_sale_from_gmt',
       'date_on_sale_to', 'date_on_sale_to_gmt', 'on_sale', 'purchasable',
       'total_sales', 'virtual', 'downloadable', 'download_limit',
       'download_expiry', 'external_url', 'button_text', 'tax_status',
       'tax_class', 'manage_stock', 'stock_quantity', 'backorders',
       'backorders_allowed', 'backordered', 'low_stock_amount',
       'sold_individually', 'weight', 'shipping_required', 'shipping_taxable',
       'shipping_class', 'shipping_class_id', 'reviews_allowed',
       'average_rating', 'rating_count', 'parent_id', 'purchase_note',
       'menu_

In [132]:
import logging

import gspread
import pandas as pd
from google.oauth2.service_account import Credentials


# Define the scope
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]

# Load credentials from service account file
creds = Credentials.from_service_account_file(
    "./opportune-study-409222-b5a62c0d49fd.json", scopes=scope
)

gc = gspread.authorize(creds)

# Create a new spreadsheet (optional, if you don't have one already)
# spreadsheet = gc.create('Product Data from Colab')
# print(f"Spreadsheet created: {spreadsheet.url}")


# Get spreadsheet by id
def get_spreadsheet(spreadsheet_id):
    try:
        return gc.open_by_key(spreadsheet_id)
    except Exception as e:
        logging.error(f"Error getting spreadsheet: {e}")
        raise

In [134]:
# from utils import update_sheet

from http import client


test_sheet_id = "1bSC1a80TZY3wRPIf07ixn8zE8g79wPn9dLt95RUho7c"
test_sheet_name = "SSI"

spreadsheet = get_spreadsheet(test_sheet_id)
stock_list = matched['Stock'].tolist()
match_sku_list = matched['SKU'].tolist()
update_sheet_cell(spreadsheet=spreadsheet, sheet_name=test_sheet_name, sku_list=match_sku_list, stock_list=stock_list)

Updated stock for SKU 9004879 at row 11
SKU 9004874 not found. Error: 'NoneType' object has no attribute 'row'
SKU 4011677 not found. Error: 'NoneType' object has no attribute 'row'


In [101]:
# stock_list = matched['stock'].tolist()
# match_sku_list = matched['sku'].tolist()
# update_sheet(spreadsheet, sheet_name, match_sku_list, stock_list)

In [84]:
test_sheet_id = "1bSC1a80TZY3wRPIf07ixn8zE8g79wPn9dLt95RUho7c"
test_sheet_name = "SSI"

In [ ]:
import gspread
# from oauth2client.service_account import ServiceAccountCredentials


In [ ]:
# import util.gsheet as gsheet

# matched_skus = [sku for sku in data['SKU'] if sku in sheet['SKU']]
# gsheet.update_sheet(spreadsheet, sheet_name, matched_skus)

In [ ]:
# import pandas as pd
# sdata['SKU_in_sheet'] = sdata['SKU'].isin(data['sku'])
# sdata['name in sheet'] = sdata['Name'].isin(data['name'])

In [57]:
# print(sdata["SKU_in_sheet"])
# print(sdata["name in sheet"])

In [23]:
# check if sdata in data or not using SKU 
# call it for enum SKU
# make a function for it

# for i in sdata:
#     if i in data['SKU']:
#         print(f"{i} exists in data")
#     else:
#         print("")